In [ ]:
%set_env PYTORCH_ENABLE_MPS_FALLBACK=1

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utilsforecast.losses import *
from utilsforecast.evaluation import evaluate

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS, NBEATSx
from neuralforecast.losses.pytorch import *
from neuralforecast.tsdataset import TimeSeriesDataset

warnings.filterwarnings("ignore")
os.environ["NIXTLA_ID_AS_COL"] = "true"
pd.set_option('display.precision', 3)

In [ ]:
url = "https://raw.githubusercontent.com/marcopeix/TimeSeriesForecastingUsingFoundationModels/refs/heads/main/data/walmart_sales_small.csv"

df = pd.read_csv(url, parse_dates=["Date"])
df = df.rename(columns={"Store": "unique_id", "Date":"ds", "Weekly_Sales":"y"})
df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['unique_id'] == i+1]

    ax.plot(plot_df['ds'], plot_df['y'])
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

## NBEATS

In [ ]:
# NBEATS

nf = NeuralForecast(models=models, freq='W')

cv_df = nf.cross_validation(df, n_windows=10, step_size=8)
cv_df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['unique_id'] == i+1]
    cv_plot_df = cv_df[cv_df['unique_id'] == i+1]

    ax.plot(plot_df['ds'], plot_df['y'])
    ax.plot(cv_plot_df['ds'], cv_plot_df['NBEATS'], ls='--', label='NBEATS')
    ax.legend()
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
eval_df = cv_df.drop(['ds', 'cutoff'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae, smape])
evaluation = evaluation.drop(['unique_id'], axis=1).groupby('metric').mean().reset_index()
evaluation

### Interpretable NBEATS

In [ ]:
Y_df = df.query("unique_id == 1")
Y_train_df = Y_df[:-32]
Y_test_df = Y_df[-32:]

# Interpretable NBEATS


nf = NeuralForecast(models=models, freq='W')
nf.fit(df=Y_train_df)

In [ ]:
model = nf.models[0]
dataset, *_ = TimeSeriesDataset.from_df(df=Y_train_df)
y_hat = model.decompose(dataset=dataset)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 7))

ax[0].plot(Y_test_df['y'].values, label='True', linewidth=4)
ax[0].plot(y_hat.sum(axis=1).flatten(), label='Forecast', color="#7B3841")
ax[0].legend()
ax[0].set_ylabel('Harmonic Signal')

ax[1].plot(y_hat[0,1]+y_hat[0,0], label='stack1', color="green")
ax[1].set_ylabel('NBEATSx Trend Stack')

ax[2].plot(y_hat[0,2], label='stack2', color="orange")
ax[2].set_ylabel('NBEATSx Seasonality Stack')
ax[2].set_xlabel(r'Prediction $\tau \in \{t+1,..., t+H\}$')
plt.show()


## NHITS

In [ ]:
# NHITS

nf = NeuralForecast(models=models, freq='W')

cv_df = nf.cross_validation(df, n_windows=10, step_size=8)
cv_df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['unique_id'] == i+1]
    cv_plot_df = cv_df[cv_df['unique_id'] == i+1]

    ax.plot(plot_df['ds'], plot_df['y'])
    ax.plot(cv_plot_df['ds'], cv_plot_df['NHITS'], ls='--', label='NHITS')
    ax.legend()
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
eval_df = cv_df.drop(['ds', 'cutoff'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae, smape])
evaluation = evaluation.drop(['unique_id'], axis=1).groupby('metric').mean().reset_index()
evaluation

### Decomposable NHITS

In [ ]:
# Decomposable NHITS


nf = NeuralForecast(models=models, freq='W')
nf.fit(df=Y_train_df)

In [ ]:
model = nf.models[0]
dataset, *_ = TimeSeriesDataset.from_df(df = Y_train_df)
y_hat = model.decompose(dataset=dataset)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 7))

ax[0].plot(Y_test_df['y'].values, label='True', linewidth=4)
ax[0].plot(y_hat.sum(axis=1).flatten(), label='Forecast', color="#7B3841")
ax[0].legend()
ax[0].set_ylabel('Harmonic Signal')

ax[1].plot(y_hat[0,1]+y_hat[0,0], label='stack1', color="green")
ax[1].set_ylabel('NHITS Stack 1')

ax[2].plot(y_hat[0,2], label='stack2', color="orange")
ax[2].set_ylabel('NHITS Stack 2')
ax[2].set_xlabel(r'Prediction $\tau \in \{t+1,..., t+H\}$')
plt.show()

## Probabilistic forecasting

In [ ]:
# Probabilistic NHITS

nf = NeuralForecast(models=models, freq='W')

nf.fit(df=Y_train_df)
preds = nf.predict()
preds.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(Y_test_df['ds'], Y_test_df['y'])
ax.plot(preds['ds'], preds['NHITS-median'], ls='--', label='NHITS')
ax.fill_between(preds['ds'], preds['NHITS-lo-90'], preds['NHITS-hi-90'], color='green', alpha=0.2)
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

## Forecasting with exogenous features

In [ ]:
futr_exog_df = nf.make_future_dataframe()
futr_exog_df['Holiday_Flag'] = Y_test_df['Holiday_Flag'].values

horizon = 32 
models = [
    NHITS(
        h = horizon,
        input_size = 2*horizon,
        # Exog features
        scaler_type = 'robust'),
    NBEATSx(
        h = horizon,
        input_size = 2*horizon,
        # Exog features
        scaler_type = 'robust',)
]

nf = NeuralForecast(models=models, freq='W')

nf.fit(df=Y_train_df)
futr_exog_df = nf.make_future_dataframe()
futr_exog_df['Holiday_Flag'] = Y_test_df['Holiday_Flag'].values
preds = nf.predict(futr_df=futr_exog_df)
preds.head()


In [ ]:
fig, ax = plt.subplots()

ax.plot(preds['ds'], Y_test_df['y'])
ax.plot(preds['ds'], preds['NHITS'], ls='--', label='NHITS')
ax.plot(preds['ds'], preds['NBEATSx'], ls=':', label='NBEATSx')
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
preds['y'] = Y_test_df['y'].values

evaluation = evaluate(df=preds, metrics=[mae, smape])
evaluation

## Transfer learning

In [ ]:
from datasetsforecast.m4 import M4

Y_df, _, _ = M4.load(directory="./", group="Hourly")
Y_df["ds"] = Y_df.groupby("unique_id")["ds"].transform(
    lambda x: pd.date_range(start="1970-01-01", periods=len(x), freq="h")
)
Y_df.head()

In [ ]:
horizon = 48
stacks = 3
models = [
    NHITS(
        input_size=5 * horizon,
        h=horizon,
        max_steps=1000,
        stack_types=stacks * ["identity"],
        n_blocks=stacks * [1],
        mlp_units=[[256, 256] for _ in range(stacks)],
        n_pool_kernel_size=stacks * [1],
        batch_size=32,
        scaler_type="standard",
        n_freq_downsample=[12, 4, 1],
    )
]
nf = NeuralForecast(models=models, freq="h")
nf.fit(df=Y_df)

In [ ]:
data_url = 'https://raw.githubusercontent.com/panambY/Hourly_Energy_Consumption/master/data/PJM_Load_hourly.csv'
df = pd.read_csv(data_url, parse_dates=['Datetime'])
df.columns = ['ds', 'y']
df.insert(0, 'unique_id', 'PJM')
df['ds'] = pd.to_datetime(df['ds'])
df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)
df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-700:], df['y'][-700:])
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
# Make input dataframe

preds = nf.predict(df=input_df)
preds.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-300:], df['y'][-300:])
ax.plot(preds['ds'], preds['NHITS'], ls='--', label='NHITS')
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
horizon = 48
stacks = 3
models = [
    NHITS(
        input_size=5 * horizon,
        h=horizon,
        max_steps=1000,
        stack_types=stacks * ["identity"],
        n_blocks=stacks * [1],
        mlp_units=[[256, 256] for _ in range(stacks)],
        n_pool_kernel_size=stacks * [1],
        batch_size=32,
        scaler_type="standard",
        n_freq_downsample=[12, 4, 1],
    )
]
nf = NeuralForecast(models=models, freq="h")
nf.fit(df=input_df)
preds_trained = nf.predict()
preds_trained.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(df['ds'][-300:], df['y'][-300:])
ax.plot(preds['ds'], preds['NHITS'], ls='--', label='NHITS - pretrained')
ax.plot(preds_trained['ds'], preds_trained['NHITS'], ls=':', label='NHITS')
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
test_df = df[-48:]
test_df['NHITS-pretrained'] = preds['NHITS'].values
test_df['NHITS'] = preds_trained['NHITS'].values
test_df.head()

In [ ]:
evaluation = evaluate(df=test_df, metrics=[mae, smape])
evaluation

## In-sample predictions

In [ ]:
# In-sample preds

In [ ]:
fig, ax = plt.subplots()

ax.plot(input_df['ds'][-300:], input_df['y'][-300:])
ax.plot(insample_preds['ds'][-300:], insample_preds['NHITS'][-300:], ls='--', label='NHITS')
ax.set_xlabel('Date')
ax.set_ylabel('Electricity consumption (MW)')
ax.set_title('Hourly electricity consumption')
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

## Hyperparameter optimization

In [ ]:
from neuralforecast.utils import AirPassengersDF

Y_df = AirPassengersDF
Y_df.head()

In [ ]:
fig, ax = plt.subplots()

ax.plot(Y_df['ds'], Y_df['y'])
ax.set_xlabel('Date')
ax.set_ylabel('Number of air passengers')
ax.legend()

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
from ray import tune
from neuralforecast.auto import AutoNHITS

In [ ]:
# NHITS config

In [ ]:
# Initialize auto model

In [ ]:
nf = NeuralForecast(models=[model], freq='M')
nf.fit(df=Y_df, val_size=24)

In [ ]:
# Get trials info

config_cols = [col for col in results.columns if col.startswith('config')]
columns_to_keep = ['loss', 'train_loss'] + config_cols
existing_columns = [col for col in columns_to_keep if col in results.columns]
filtered_results = results[existing_columns]
best_runs = filtered_results.sort_values('loss', ascending=True).head(5)
best_runs